In [1]:
import pandas as pd
import dvc.api as DvcApi
import cohere

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import os
from dotenv import load_dotenv #, find_dotenv

In [4]:
# dotenv_path = find_dotenv(usecwd=True)
# load_dotenv(dotenv_path ,encoding='utf-16')
load_dotenv(encoding='utf-16')

api_key = os.getenv('apikey')

In [6]:
# Paste your API key here. Remember to not share publicly

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [35]:
pd.set_option('display.max_colwidth', None)

# Named Entity Recognition (NER)  with Generative Model of Cohere

In this notebook we use Cohere's generative models to extract entities from a job description. We make use of sturctured generation based on providing multiple examples in the prompt.

The data are job descriptions (together named entities) and relationships between entities in json format. In this preliminary work we will not consider the relationships as NER alone is not enough to achieve this.

- Dataset 1: For development and training
- Dataset 2: For testing and final reporting


## Preparing examples for the prompt

In our prompt, we'll present the model with examples of job descriptions and the type of output we're after.

In [8]:
path = "data/relations_dev.json"
repo = "../"
version = "v2"

data_url = DvcApi.get_url(path = path, repo = repo, rev = version) #could be tag or git commit
data_dev = pd.read_json(data_url)

In [9]:
data_dev.head(2)

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG..."
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."


In [63]:
def fun(x):
    y = 'Extracted Text:\n'
    if len(x) != 0:
        for i in x:
            text = i['text']
            entityLabel = i['entityLabel']
            y = y+entityLabel+': '+text+'\n'
            
    return y

In [64]:
data_dev.tokens.apply(lambda x: fun(x))

0                                                                                                                                                                                                  Extracted Text:\nDIPLOMA: Bachelor\nDIPLOMA_MAJOR: Mechanical Engineering\nDIPLOMA_MAJOR: Physical Science\nEXPERIENCE: 3+ years\nSKILLS: developing\nSKILLS: fiber optic cables\nSKILLS: connector related products\n
1                                                                                                                                                                     Extracted Text:\nEXPERIENCE: 10+ years\nSKILLS: software engineering\nEXPERIENCE: 5+ years\nSKILLS: technical management\nSKILLS: release engineering\nSKILLS: tools engineering\nSKILLS: DevOps\nDIPLOMA: BS/MS\nDIPLOMA_MAJOR: Computer Science\n
2                                                                                                                                                                                   

In [79]:
prompt = ''
for i in range(7):
    prompt = prompt + 'DOCUMENT: ' + data_dev.iloc[i].document +'\n'+fun(data_dev.iloc[i].tokens) + '--\n'

In [80]:
example_prompt = 'DOCUMENT: ' + data_dev.iloc[7].document + '\n' + 'Extracted Text:'

In [81]:
"""
def extract(self, example):
    extraction = co.generate(model='large', prompt=self.make_prompt(example), max_tokens=10, 
                             temperature=0.1, stop_sequences=["\n"])
    return(extraction.generations[0].text[:-1])
"""
    
extraction = co.generate( model = 'large', prompt = prompt + example_prompt, max_tokens = 100, temperature = 0.1,
                        stop_sequences = ["--"])

In [82]:
extraction

cohere.Generations {
	generations: [cohere.Generation {
	text: 
EXPERIENCE: 5+ years
SKILLS: backend services
EXPERIENCE: 4+ years
SKILLS: systems
SKILLS: data infrastructure
--
	likelihood: None
	token_likelihoods: None
}]
	return_likelihoods: NONE
}

In [83]:
extraction.generations[0]

cohere.Generation {
	text: 
EXPERIENCE: 5+ years
SKILLS: backend services
EXPERIENCE: 4+ years
SKILLS: systems
SKILLS: data infrastructure
--
	likelihood: None
	token_likelihoods: None
}

In [84]:
print(fun(data_dev.iloc[7].tokens))

Extracted Text:
DIPLOMA: BS/MS
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 5+ years
SKILLS: architecting
SKILLS: developing
EXPERIENCE: 4+ years
SKILLS: managing cross-functional engineering teams

